# Graph neat examples in Raku

### ***Set 2***

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
July, November 2024   

In [ ]:
#% js
 use Graph::TriangularGrid;
 use JavaScript::D3;
 
 my $g = Graph::TriangularGrid.new(4, 4);
 my @highlight = ($g.vertex-list Z=> $g.vertex-degree).classify(*.value).map({ $_.value».key });


  js-d3-graph-plot( $g.edges(:dataset),
        :@highlight,
        background => '#1F1F1F', 
        edge-thickness => 3,
        vertex-size => 10,
        vertex-label-color => 'none',
        width => 1000,
        height => 300, 
        margins => 5,
        edge-color => 'SteelBlue',
        force => {charge => {strength => -300}, y => {strength => 0.2}, link => {minDistance => 4}}
    ) 

In [ ]:
#% js
  use Math::SparseMatrix;
  use Data::Reshapers;

  my ($amat, $imat) = $g.adjacency-matrix, $g.incidence-matrix;
  
  my %opts = grid-lines => {width => 0.25, color => 'DimGray'}, title-color => 'Silver', color-palette => 'Inferno', :!tooltip, background => '#1F1F1F', height => 300;

  js-d3-matrix-plot($amat, plot-label => 'Adjacency matrix', |%opts, width => 300+80, margins => {:2left, :40top, :2bottom, :80right})
  ~ "\n" ~ 
  js-d3-matrix-plot($imat, plot-label => 'Incidence matrix', |%opts, width => 600, margins => {:2left, :40top, :2bottom, :2right})

----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All computational graph features discussed here are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).   
- Graph plotting with:
    - `js-d3-graph-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
    -  `Graph.dot`, that makes SVG images via [Graphviz](https://graphviz.org).

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [ ]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::HexagonalGrid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Path;
use Graph::Petersen;
use Graph::Star;
use Graph::TriangularGrid;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Text::Levenshtein::Damerau;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Silver';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

--------

## Bipartite graph coloring

*Maybe this example is not that neat, since it might see too regular or basic.*    
*But it is a good warm up.*

Make grid graph:

In [ ]:
my $gg = Graph::Grid.new(6, 16);

In [ ]:
$gg.is-bipartite

Show bipartite coloring of the grid graph:

In [ ]:
$gg.bipartite-coloring

Prepare graph highlight code:

In [ ]:
my %highlight = <SlateBlue Orange> Z=> $gg.bipartite-coloring.classify(*.value).nodemap(*».key).values;

.say for %highlight

Plot grid graph as a bipartite graph:

In [ ]:
#%js
$gg.edges(:dataset) ==>
    js-d3-graph-plot(
        :%highlight,
        vertex-coordinates => $gg.vertex-coordinates,
        :$background, :$title-color,
        edge-thickness => 5,
        vertex-size => 12,
        vertex-label-color => 'none',
        directed => $g.directed,
        title => 'Grid graph bipartite coloring', 
        width => 1000,
        height => 400, 
        margins => {top => 80},
        force => {charge => {strength => -300}, x => {strength => 0.12}, link => {minDistance => 4}}
        )

------

## Highlight connected components

Make a grid graph with randomly directed edges:

In [ ]:
my $g = Graph::Grid.new(10, 20, :!directed);
my $g2 = $g.directed-graph(method => 'random');

Find the connected components in the graph:

In [ ]:
my @components = $g2.connected-components.grep(*.elems - 1);
@components».elems;

Highlight the connected components in a graph:

In [ ]:
#% js
$g2.edges(:dataset) ==> 
js-d3-graph-plot(
    vertex-coordinates => $g2.vertex-coordinates,
    highlight => @components,
    directed => $g2.directed,
    :$background, 
    :$title-color, 
    width => 1000, 
    height => 500, 
    vertex-label-color => 'none',
    vertex-fill-color => 'SteelBlue',
    :$edge-thickness,
    edge-color => 'Gray',
    vertex-size => 14,
    arrowhead-size => 4,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
)

### Visualize via ***Graphviz DOT***

Make a triagular grid graph:

In [ ]:
my $g3 = Graph::TriangularGrid.new(8, 16, scale => 0.3, :!directed);
$g3 = $g3.directed-graph(method => 'random', flip-threshold => 0.25);

Visualize using the [Graphviz DOT language](https://graphviz.org/doc/info/lang.html):

In [ ]:
#% html
$g3.dot( 
    highlight => $g3.connected-components.grep(*.elems - 1),
    :!node-labels,
    node-shape => <hexagon triangle>.pick, 
    node-height => 0.7, 
    node-width => 0.7, 
    edge-thickness => 4, 
    edge-color => 'Gray',
    size => '10,6!',
    engine => 'neato' 
):svg

---------

## Collage of star graphs

In [ ]:
my @graphs = (^40).map({ Graph::Star.new(n => (8..16).pick, prefix => "$_-") });
my $bigGraph = Graph.new( @graphs.map(*.edges).flat )

Get a range of colors:

In [ ]:
my @colors = (^14).map: { sprintf "#%02X%02X%02X", 250 - $_*10, 128 - $_*5, 114 + $_*10 };

Plot the collage of graphs:

In [204]:
#%js
$bigGraph.edges(:dataset) ==>
js-d3-graph-plot(
    highlight => (@colors Z=> @graphs.pick(@colors.elems).map({ [|$_.vertex-list, |$_.edges] })).Hash,
    :$background, :$title-color,
    edge-thickness => 2, 
    vertex-stroke-color => 'LightSteelBlue',
    vertex-size => 8,
    vertex-label-color => 'none',
    directed => False,
    title => 'Collage graph', 
    width => 1200,
    height => 700, 
    force => {charge => {strength => -40}, y => {strength => 0.2}, collision => {radius => 12}, link => {minDistance => 4}}
    )

### Visualize via Graphviz DOT

In [ ]:
#% html
my $preamble = q:to/END/;
label = "Collage graph";
labelloc = "b";
fontcolor = "Gray";
fontsize = 42;
bgcolor = "#1F1F1F";
graph [size="9,9!"];
node [label="", shape=circle, style="filled", fillcolor="SteelBlue", color="Ivory", penwidth=3, width=0.65, height=0.65];
edge [color="SteelBlue", penwidth=4]
END

$bigGraph.dot(
    highlight => (@colors Z=> @graphs.pick(@colors.elems).map({ [|$_.vertex-list, |$_.edges] }) ).Hash,
    :$preamble,
    engine => 'neato'):svg